In [40]:
import pandas as pd
import numpy as np
import catboost
from sklearn import cross_validation

In [6]:
data = pd.read_csv('train.tsv', error_bad_lines=False, delimiter='\t')

In [9]:
data.head(5)

,Unnamed: 0,address,candidate_num,group_num,has_wifi,lat,lon,names,publishing_status,rubrics,ssid,target,urls,user_lat,user_lon
0,0,"Россия, Новосибирская область, Обь, проспект М...",0,0,NaN,55.007759,82.667023,"[""Сбербанк России, банкомат"", ""Sberbank Rossii...",publish,[30336],Tolmachevo-MTS-Free,0,"[{""type"": ""main"", ""value"": ""http://www.sberban...",55.007579,82.666723
1,1,"Россия, Новосибирская область, аэропорт Новоси...",1,0,NaN,55.007879,82.665401,"[""Ил-86""]",publish,[3481524327],Tolmachevo-MTS-Free,0,[],55.007579,82.666723
2,2,"Россия, Новосибирская область, Обь, проспект М...",2,0,NaN,55.007570,82.667069,"[""Чашка кофе"", ""Чашка Кофе""]",publish,[31495],Tolmachevo-MTS-Free,0,"[{""type"": ""mining"", ""value"": ""https://2gis.ru/...",55.007579,82.666723
3,3,"Россия, Новосибирская область, Обь, проспект М...",3,0,True,55.007337,82.667066,"[""Чашка кофе"", ""Chashka kofe"", ""Чашка кофе"", ""...",publish,[31495],Tolmachevo-MTS-Free,0,"[{""type"": ""main"", ""value"": ""http://chashkacoff...",55.007579,82.666723
4,4,"Россия, Новосибирская область, Обь, проспект М...",4,0,NaN,55.007623,82.666341,"[""Телефон доверия""]",obsolete,[30078],Tolmachevo-MTS-Free,0,"[{""type"": ""mining"", ""value"": ""https://2gis.ru/...",55.007579,82.666723


In [11]:
data.shape

(95067, 15)

In [76]:
data['publishing_status'].describe()

count       95067
unique          8
top       publish
freq        42004
Name: publishing_status, dtype: object

In [26]:
data_2 = pd.DataFrame(columns=[data['lat'], data['lon']])


In [32]:
data_2 = data.drop(columns=['Unnamed: 0', 'address', 'candidate_num', 'group_num', 'has_wifi', 'names', 'publishing_status',
                            'rubrics', 'ssid', 'urls'])

In [36]:
data_2['target'].describe()

count    95067.000000
mean         0.055224
std          0.228419
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: target, dtype: float64

In [38]:
y = data_2['target']
X = data_2.drop(columns='target')

In [43]:
estimator = catboost.CatBoostClassifier(verbose=False)
estimator.fit(X, y)

In [45]:
cross_validation.cross_val_score(estimator, X, y, cv=5, n_jobs=4 )

array([0.94788051, 0.94877459, 0.94566875, 0.91879241, 0.93756903])

In [55]:
data_has = data['has_wifi']
data_has[data_has == True] = 2
data_has[data_has == False] = 3
data_has = data['has_wifi'].fillna(1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
data_has

0        1
1        1
2        1
3        2
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
95037    1
95038    1
95039    1
95040    1
95041    1
95042    1
95043    1
95044    1
95045    1
95046    1
95047    1
95048    1
95049    1
95050    1
95051    1
95052    1
95053    1
95054    1
95055    1
95056    1
95057    1
95058    1
95059    1
95060    1
95061    1
95062    1
95063    1
95064    1
95065    1
95066    1
Name: has_wifi, Length: 95067, dtype: int64

In [58]:
X['has_wifi'] = data_has

In [59]:
X.head(5)

,lat,lon,user_lat,user_lon,has_wifi
0,55.007759,82.667023,55.007579,82.666723,1
1,55.007879,82.665401,55.007579,82.666723,1
2,55.007570,82.667069,55.007579,82.666723,1
3,55.007337,82.667066,55.007579,82.666723,2
4,55.007623,82.666341,55.007579,82.666723,1


In [67]:
pool = catboost.Pool(X, y, cat_features=[4])

In [68]:
estimator_has = catboost.CatBoostClassifier(verbose=False)
estimator_has.fit(pool)

In [ ]:
np.mean(cross_validation.cross_val_score(estimator, X, y, cv=5, n_jobs=4 ))

In [75]:
catboost.cv(pool, fold_count=5, params=params, iterations=100)

0:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 379ms	remaining: 37.5s
1:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 737ms	remaining: 36.1s
2:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 1.03s	remaining: 33.3s
3:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 1.35s	remaining: 32.4s
4:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 1.66s	remaining: 31.6s
5:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 1.95s	remaining: 30.5s
6:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 2.3s	remaining: 30.6s
7:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 2.63s	remaining: 30.2s
8:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 2.93s	remaining: 29.7s
9:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 3.22s	remaining: 29s
10:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)	total: 3.53s	remaining: 28.6s
11:	learn: 0.9447758	test: 0.9447758	best: 0.9447758 (0)

95:	learn: 0.9450230	test: 0.9447968	best: 0.9448284 (91)	total: 23.9s	remaining: 998ms
96:	learn: 0.9450335	test: 0.9448284	best: 0.9448284 (91)	total: 24.1s	remaining: 747ms
97:	learn: 0.9450440	test: 0.9448284	best: 0.9448284 (91)	total: 24.4s	remaining: 498ms
98:	learn: 0.9450467	test: 0.9448179	best: 0.9448284 (91)	total: 24.7s	remaining: 249ms
99:	learn: 0.9450598	test: 0.9448705	best: 0.9448705 (99)	total: 24.9s	remaining: 0us


,test-Accuracy-mean,test-Accuracy-std,test-Logloss-mean,test-Logloss-std,train-Accuracy-mean,train-Accuracy-std,train-Logloss-mean,train-Logloss-std
0,0.944776,0.001059,0.656141,0.000045,0.944776,0.000265,0.656134,0.000079
1,0.944776,0.001059,0.621988,0.000077,0.944776,0.000265,0.621972,0.000156
2,0.944776,0.001059,0.590477,0.000115,0.944776,0.000265,0.590451,0.000225
3,0.944776,0.001059,0.561425,0.000158,0.944776,0.000265,0.561390,0.000285
4,0.944776,0.001059,0.534648,0.000203,0.944776,0.000265,0.534609,0.000339
5,0.944776,0.001059,0.509983,0.000245,0.944776,0.000265,0.509939,0.000394
6,0.944776,0.001059,0.487268,0.000294,0.944776,0.000265,0.487218,0.000438
7,0.944776,0.001059,0.466352,0.000347,0.944776,0.000265,0.466295,0.000479
8,0.944776,0.001059,0.447091,0.000397,0.944776,0.000265,0.447032,0.000516
9,0.944776,0.001059,0.429363,0.000449,0.944776,0.000265,0.429297,0.000547


In [73]:
params = {'loss_function':'Logloss', "eval_metric":'Accuracy'}